In [32]:
# Import Required Libraries
import pandas as pd
import numpy as np
import joblib
import json
import sklearn  # To check scikit-learn version compatibility

In [33]:
# Function to Load Dataset
def load_dataset(file_path):
    try:
        data = pd.read_csv(file_path)
        print("Dataset loaded successfully!\n")
        return data
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")
        return None
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

In [34]:
# Function to Get Student Data
def get_student_data(registration_number, data):
    student = data[data["Reg. No."].astype(str).str.strip() == registration_number.strip()]
    if student.empty:
        print(f"Error: Student with Registration Number '{registration_number}' not found.\n")
        return None
    return student.iloc[0]  # Extract first matching row as a Series

In [35]:
# Function to Load Model, Scaler, and Metadata
def load_model_and_metadata(model_path, scaler_path, metadata_path):
    try:
        # Load Metadata
        with open(metadata_path, "r") as f:
            metadata = json.load(f)

        # Check scikit-learn version compatibility
        current_version = sklearn.__version__
        saved_version = metadata.get("scikit-learn_version", "Unknown")

        if current_version != saved_version:
            print(f"Warning: scikit-learn version mismatch! (Saved: {saved_version}, Current: {current_version})")
            print("This may cause unexpected behavior.\n")

        # Load Model and Scaler
        model = joblib.load(model_path)
        scaler = joblib.load(scaler_path)
        return model, scaler, metadata
    except FileNotFoundError:
        print("Error: Model, scaler, or metadata file not found.")
        return None, None, None
    except Exception as e:
        print(f"Error loading model or metadata: {e}")
        return None, None, None

In [36]:
# Function to Predict Probation Probability
def predict_probation(student_row, model, scaler, metadata):
    try:
        # Prepare Input Features using the feature indices stored in metadata
        feature_indices = metadata["feature_indices"]  # Feature indices
        input_features = np.array([[student_row.iloc[index] for index in feature_indices]])

        # Scale Features
        input_features_scaled = scaler.transform(input_features)
        probation_percentage = model.predict(input_features_scaled)[0]
        return round(probation_percentage, 2)
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None

In [37]:
# Function to Display Student Report
def display_student_report(student_row, probation_percentage):
    print("\n--- Student Report ---\n")
    print(f"Student Name: {student_row['Student Name']}")
    print(f"Registration Number: {student_row['Reg. No.']}")
    print(f"Probation Status: {student_row['Current Probation Status (0-2)']}")
    print(f"CGPA: {student_row['Current CGPA (4.00)']}\n")

    # Display Midterm Performance
    print("Performance Till Midterm Exams:\n")
    print(f"{'S.#':<5} {'Course Name':<60} {'Marks - 45':<15} {'Pass / Fail':<10}")

    # Filter Course Columns
    excluded_columns = ["Previous Semester GPA (4.00)", "Current CGPA (4.00)", "Current Probation Status (0-2)", "Probation Chance (Out of 100%)",
                        "Contacts", "< >", "< >.1", "< >.2"]
    course_columns = [col for col in student_row.index if col not in excluded_columns and col not in ['Student Name', 'Reg. No.']]

    # Display Results
    for i, course in enumerate(course_columns, start=1):
        marks = student_row[course]
        status = "Pass" if marks >= 24 else "Fail"
        print(f"{i:<5} {course:<60} {marks:<15} {status:<10}")

    # Recommendation
    print("\nRecommendation:")
    if probation_percentage >= 50:
        print(f"Your chance of probation is {probation_percentage}%. Consider withdrawing subjects with low marks and focusing on prerequisites.")
    elif probation_percentage >= 30:
        print(f"Your chance of probation is {probation_percentage}%. Work harder for final exams and focus on weak subjects.")
    else:
        print(f"Your chance of probation is {probation_percentage}%. Keep up the good work and maintain your performance.\n")

In [40]:
# Main Execution Function
def main():
    # File Paths
    dataset_path = "test-dataset.csv"  # Update with your file path
    model_path = "probation_rf_model.pkl"  # Update with Random Forest model path
    scaler_path = "scaler.pkl"  # Update with scaler path
    metadata_path = "metadata.json"  # Metadata file

    # Load Dataset
    data = load_dataset(dataset_path)
    if data is None:
        return

    # Load Model, Scaler, and Metadata
    model, scaler, metadata = load_model_and_metadata(model_path, scaler_path, metadata_path)
    if model is None or scaler is None or metadata is None:
        return

    # Input Registration Number
    registration_number = input("Enter Student Registration Number: ").strip()
    student_row = get_student_data(registration_number, data)
    if student_row is None:
        return

    # Predict Probation Probability
    probation_percentage = predict_probation(student_row, model, scaler, metadata)
    if probation_percentage is None:
        return

    # Display Report
    display_student_report(student_row, probation_percentage)

In [41]:
# Run the Program
if __name__ == "__main__":
    main()

Dataset loaded successfully!

Enter Student Registration Number: 21108106

--- Student Report ---

Student Name: Maryam
Registration Number: 21108106
Probation Status: 0
CGPA: 2.94

Performance Till Midterm Exams:

S.#   Course Name                                                  Marks - 45      Pass / Fail
1     Mathematics                                                  28              Pass      
2     Quantum Physics                                              32              Pass      
3     Biological Theory                                            35              Pass      
4     Assembly Language                                            32              Pass      
5     General Knowledge                                            31              Pass      
6     Meta Psychology                                              32              Pass      
7     Neural Networks                                              33              Pass      
8     Islamic Studies           

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
